# 480. Sliding Window Median

The median is the middle value in an ordered integer list. If the size of the list is even, there is no middle value. So the median is the mean of the two middle values.For examples, if arr = [2,3,4], the median is 3.For examples, if arr = [1,2,3,4], the median is (2 + 3) / 2 = 2.5.You are given an integer array nums and an integer k. There is a sliding window of size k which is moving from the very left of the array to the very right. You can only see the k numbers in the window. Each time the sliding window moves right by one position.Return the median array for each window in the original array. Answers within 10-5 of the actual value will be accepted. **Example 1:**Input: nums = [1,3,-1,-3,5,3,6,7], k = 3Output: [1.00000,-1.00000,-1.00000,3.00000,5.00000,6.00000]Explanation: Window position                Median---------------                -----[1  3  -1] -3  5  3  6  7        1 1 [3  -1  -3] 5  3  6  7       -1 1  3 [-1  -3  5] 3  6  7       -1 1  3  -1 [-3  5  3] 6  7        3 1  3  -1  -3 [5  3  6] 7        5 1  3  -1  -3  5 [3  6  7]       6**Example 2:**Input: nums = [1,2,3,4,2,3,1,4,2], k = 3Output: [2.00000,3.00000,3.00000,3.00000,2.00000,3.00000,2.00000] **Constraints:**1 <= k <= nums.length <= 105-231 <= nums[i] <= 231 - 1

## Solution Explanation
This problem asks us to find the median of each sliding window of size k in an array. The key challenge is to efficiently maintain a sorted structure as the window slides.A naive approach would be to extract each window, sort it, and find the median. However, this would be inefficient with O(k log k) time for each window.Instead, we can use two heaps to track the median efficiently:1. A max heap for the smaller half of elements2. A min heap for the larger half of elementsThe heaps should be balanced (or nearly balanced) so that:* If k is odd, the max heap has one more element than the min heap, and the median is the top of the max heap.* If k is even, both heaps have the same number of elements, and the median is the average of the tops of both heaps.As the window slides, we need to:1. Add the new element to the appropriate heap2. Remove the element that's no longer in the window3. Rebalance the heaps if necessarySince Python's heapq doesn't support removal from the middle of a heap efficiently, we'll use a "lazy removal" approach with a hash map to track elements that should be removed.

In [ ]:
import heapqfrom collections import defaultdictdef medianSlidingWindow(nums, k):    result = []        # Use two heaps to track the median    small = []  # max heap for smaller half (negate values to simulate max heap)    large = []  # min heap for larger half        # Hash map to track elements to be removed    to_remove = defaultdict(int)        # Helper function to remove elements from heap    def remove_outdated(heap, is_small):        while heap and to_remove[heap[0] * (-1 if is_small else 1)] > 0:            element = heapq.heappop(heap)            element = element * (-1 if is_small else 1)  # Convert back to original value            to_remove[element] -= 1        # Helper function to rebalance heaps    def rebalance():        # Ensure small has either the same number of elements as large, or one more        if len(small) > len(large) + 1:            # Move one element from small to large            val = -heapq.heappop(small)            heapq.heappush(large, val)        elif len(large) > len(small):            # Move one element from large to small            val = heapq.heappop(large)            heapq.heappush(small, -val)        # Process the first k elements    for i in range(k):        heapq.heappush(small, -nums[i])        # Rebalance to move half of the elements to large heap    for _ in range(k // 2):        heapq.heappush(large, -heapq.heappop(small))        # Process all windows    for i in range(k, len(nums) + 1):        # Calculate median for current window        if k % 2 == 1:            # Odd window size: median is top of small heap            median = -small[0]        else:            # Even window size: median is average of tops of both heaps            median = (-small[0] + large[0]) / 2                if i < len(nums):            result.append(median)                        # Remove the leftmost element of the window            outgoing = nums[i - k]            # Mark for removal            to_remove[outgoing] += 1                        # Adjust heap sizes            if outgoing <= -small[0]:                # Outgoing element is in small heap                if outgoing == -small[0]:                    remove_outdated(small, True)                else:                    # Will be removed later                    pass            else:                # Outgoing element is in large heap                if outgoing == large[0]:                    remove_outdated(large, False)                else:                    # Will be removed later                    pass                        # Add the new element            incoming = nums[i]            if small and incoming <= -small[0]:                heapq.heappush(small, -incoming)            else:                heapq.heappush(large, incoming)                        # Clean up outdated elements and rebalance            remove_outdated(small, True)            remove_outdated(large, False)            rebalance()        else:            result.append(median)        return result

## Time and Space Complexity
* *Time Complexity**: O(n log k) where n is the length of the input array and k is the window size.* For each of the n-k+1 windows, we perform heap operations (push, pop) which take O(log k) time.* The lazy removal approach ensures that each element is pushed and popped at most once, keeping the overall complexity at O(n log k).* *Space Complexity**: O(k + n) where:* O(k) for the two heaps which store at most k elements combined* O(n) in the worst case for the hash map that tracks elements to be removed (if all elements are unique)

## Test Cases


In [ ]:
def test_median_sliding_window():    # Test case 1: Example from the problem    nums1 = [1, 3, -1, -3, 5, 3, 6, 7]    k1 = 3    expected1 = [1.00000, -1.00000, -1.00000, 3.00000, 5.00000, 6.00000]    result1 = medianSlidingWindow(nums1, k1)    assert all(abs(a - b) < 1e-5 for a, b in zip(result1, expected1)), f"Test case 1 failed: {result1} != {expected1}"        # Test case 2: Another example from the problem    nums2 = [1, 2, 3, 4, 2, 3, 1, 4, 2]    k2 = 3    expected2 = [2.00000, 3.00000, 3.00000, 3.00000, 2.00000, 3.00000, 2.00000]    result2 = medianSlidingWindow(nums2, k2)    assert all(abs(a - b) < 1e-5 for a, b in zip(result2, expected2)), f"Test case 2 failed: {result2} != {expected2}"        # Test case 3: Window size equals array length    nums3 = [1, 2, 3, 4, 5]    k3 = 5    expected3 = [3.00000]    result3 = medianSlidingWindow(nums3, k3)    assert all(abs(a - b) < 1e-5 for a, b in zip(result3, expected3)), f"Test case 3 failed: {result3} != {expected3}"        # Test case 4: Window size 1    nums4 = [1, 2, 3, 4, 5]    k4 = 1    expected4 = [1.00000, 2.00000, 3.00000, 4.00000, 5.00000]    result4 = medianSlidingWindow(nums4, k4)    assert all(abs(a - b) < 1e-5 for a, b in zip(result4, expected4)), f"Test case 4 failed: {result4} != {expected4}"        # Test case 5: Even window size    nums5 = [1, 2, 3, 4, 5, 6]    k5 = 4    expected5 = [2.50000, 3.50000, 4.50000]    result5 = medianSlidingWindow(nums5, k5)    assert all(abs(a - b) < 1e-5 for a, b in zip(result5, expected5)), f"Test case 5 failed: {result5} != {expected5}"        print("All test cases passed!")# Run the teststest_median_sliding_window()